In [ ]:
!pip install pafy
!pip install youtube-dl 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import time

t_end = time.time() + 60

In [ ]:
import cv2
import numpy as np
import pafy
import time

from google.colab.patches import cv2_imshow
fourcc = cv2.VideoWriter_fourcc(*'XVID')
w, h = 1280,720
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
output_movie = cv2.VideoWriter('/content/Detected_Laser_Pointer2.avi', fourcc, 30, (w, h))

#url of the video to predict Age and gender
url = 'https://www.youtube.com/watch?v=iH1ZJVqJO3Y'
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="mp4")

cap = cv2.VideoCapture(play.url)

cap.set(3, 480) #set width of the frame
cap.set(4, 640) #set height of the frame

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']

gender_list = ['Male', 'Female']

def load_caffe_models():
 
  age_net = cv2.dnn.readNetFromCaffe('/content/drive/MyDrive/SoC2k21/New folder/My_PreTrained/deploy_age.prototxt', '/content/drive/MyDrive/SoC2k21/New folder/My_PreTrained/age_net.caffemodel')
  
  gender_net = cv2.dnn.readNetFromCaffe('/content/drive/MyDrive/SoC2k21/New folder/My_PreTrained/deploy_gender.prototxt', '/content/drive/MyDrive/SoC2k21/New folder/My_PreTrained/gender_net.caffemodel')

  return (age_net, gender_net)

def video_detector(age_net, gender_net):

  font = cv2.FONT_HERSHEY_SIMPLEX

  while time.time() < t_end:    

    ret, image = cap.read()
          
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
      
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2_imshow(gray)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if(len(faces)>0):
      print("Found {} faces".format(str(len(faces))))

    for (x, y, w, h )in faces:
      cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)

    #Get Face 
      face_img = image[y:y+h, h:h+w].copy()
      blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

    #Predict Gender
      gender_net.setInput(blob)
      gender_preds = gender_net.forward()
      gender = gender_list[gender_preds[0].argmax()]
      print("Gender : " + gender)

    #Predict Age
      age_net.setInput(blob)
      age_preds = age_net.forward()
      age = age_list[age_preds[0].argmax()]
      print("Age Range: " + age)

      overlay_text = "%s %s" % (gender, age)
      cv2.putText(image, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    print(np.shape(image))
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    output_movie.write(image)
    
  
    #cv2.imshow('frame', image)  
    #0xFF is a hexadecimal constant which is 11111111 in binary.
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
   
output_movie.release()
age_net, gender_net = load_caffe_models()

video_detector(age_net, gender_net)